In [3]:
import sqlite3
import pandas as pd

DB_FILE = "ntpu_paper.sqlite"

conn = sqlite3.connect(DB_FILE)
cur = conn.cursor()

# === 若留言表不存在就建立 ===
cur.execute("""
CREATE TABLE IF NOT EXISTS article_comments (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    link_id TEXT,
    article_id TEXT,
    comment_index INTEGER,
    comment_text TEXT,
    pua_status TEXT DEFAULT 'pending',
    FOREIGN KEY (article_id) REFERENCES articles(id),
    FOREIGN KEY (link_id) REFERENCES links(id)
);
""")
conn.commit()

# === 讀取含有 replies 的文章 ===
df = pd.read_sql_query("""
    SELECT id AS article_id, link_id, replies
    FROM articles
    WHERE replies IS NOT NULL AND TRIM(replies) != '';
""", conn)

print(f"📘 共讀取 {len(df)} 篇含有留言的文章，準備切割 replies...")

for _, row in df.iterrows():
    article_id = row["article_id"]
    link_id = row["link_id"]
    replies_raw = row["replies"].strip()

    # --- 切割留言 ---
    # 有些資料可能使用 '---' 分隔，有些可能用換行符；可混合處理
    if '\n\n---\n\n' in replies_raw:
        parts = [p.strip() for p in replies_raw.split('---') if p.strip()]

    # --- 寫入每則留言 ---
    for idx, comment in enumerate(parts):
        cur.execute("""
            INSERT INTO article_comments (link_id, article_id, comment_index, comment_text)
            VALUES (?, ?, ?, ?);
        """, (link_id, article_id, idx, comment))

    print(f"✅ {article_id} 已拆出 {len(parts)} 則留言")

conn.commit()
conn.close()
print("🎉 全部完成，已將 replies 拆入 article_comments 表！")

📘 共讀取 365 篇含有留言的文章，準備切割 replies...
✅ 68fce9b6af1137205015fd06 已拆出 8 則留言
✅ 68fce9f3af11377624f11eda 已拆出 7 則留言
✅ 68fcea01af11377624f11edb 已拆出 8 則留言
✅ 68fcea6eaf11379f9c929394 已拆出 8 則留言
✅ 68fcea81af11379f9c929395 已拆出 8 則留言
✅ 68fcea94af11379f9c929396 已拆出 8 則留言
✅ 68fceaa7af11379f9c929397 已拆出 7 則留言
✅ 68fceabaaf11379f9c929398 已拆出 8 則留言
✅ 68fceae1af11379f9c929399 已拆出 2 則留言
✅ 68fceaf4af11379f9c92939a 已拆出 8 則留言
✅ 68fceb90af11379f9c92939b 已拆出 7 則留言
✅ 68fcebb5af11379f9c92939c 已拆出 3 則留言
✅ 68fcec3daf11379f9c92939d 已拆出 8 則留言
✅ 68fcec76af11379f9c92939e 已拆出 2 則留言
✅ 68fcec89af11379f9c92939f 已拆出 8 則留言
✅ 68fcec9baf11379f9c9293a0 已拆出 8 則留言
✅ 68fcfa69dbeaa9ea10b3402f 已拆出 9 則留言
✅ 68fcfa9fdbeaa9ea10b34030 已拆出 7 則留言
✅ 68fcfb19dbeaa9ee9ad3699f 已拆出 8 則留言
✅ 68fcfb26dbeaa9ee9ad369a0 已拆出 2 則留言
✅ 68fcfb41dbeaa9ee9ad369a1 已拆出 6 則留言
✅ 68fcfbd4dbeaa9ee9ad369a2 已拆出 5 則留言
✅ 68fcfc4adbeaa9ee9ad369a3 已拆出 5 則留言
✅ 68fcfc65dbeaa9ee9ad369a4 已拆出 8 則留言
✅ 68fcfd0edbeaa9ee9ad369a5 已拆出 8 則留言
✅ 68fcfd8bdbeaa9ee9ad369a7 已拆出 2 則留言
✅ 6